In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import numpy as np
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [98]:
from meli.text import normalize_text, get_embeddings, extract_top_n, most_similar

In [30]:
raw_data = "../data/raw/items_titles.csv"
raw_data_test = "../data/raw/items_titles_test.csv"

processed_data = "../data/processed/items_titles_sim.csv"

In [ ]:
df = pd.read_csv(raw_data)
df_test = pd.read_csv(raw_data_test)

## Preprocessing text

In [33]:
df["normalized_text"] = df["ITE_ITEM_TITLE"].apply(normalize_text)
df_test["normalized_text"] = df_test["ITE_ITEM_TITLE"].apply(normalize_text)

In [35]:
df.head()

,ITE_ITEM_TITLE,normalized_text
0,Tênis Ascension Posh Masculino - Preto E Verme...,tênis ascension posh masculino preto vermelho
1,Tenis Para Caminhada Super Levinho Spider Corr...,tenis caminhada super levinho spider corrida
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,tênis feminino le parc hock black ice original...
3,Tênis Olympikus Esportivo Academia Nova Tendên...,tênis olympikus esportivo academia nova tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,inteligente led bicicleta tauda luz usb bicicl...


In [36]:
df_test.head()

,ITE_ITEM_TITLE,normalized_text
0,Tênis Olympikus Esporte Valente - Masculino Kids,tênis olympikus esporte valente masculino kid
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,bicicleta barra forte samy c marchas cubo c ro...
2,Tênis Usthemp Slip-on Temático - Labrador 2,tênis usthemp slip on temá tico labrador
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tênis casual feminino moleca tecido tie dye
4,Tênis Star Baby Sapatinho Conforto + Brinde,tênis star baby sapatinho conforto brinde


## Sentence encoding

#### Descrição do modelo utilizado:
[**distiluse-base-multilingual-cased-v1**](https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1): Multilingual knowledge distilled version of [multilingual Universal Sentence Encoder](https://arxiv.org/abs/1907.04307). Supports 15 languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish.

Fonte: https://www.sbert.net/docs/pretrained_models.html

In [37]:
%%time
embeddings = get_embeddings(df["normalized_text"], "embeddings")

CPU times: user 8min 45s, sys: 12.7 s, total: 8min 58s
Wall time: 5min 15s


In [38]:
%%time
embeddings_test = get_embeddings(df_test["normalized_text"], "embeddings_test")

CPU times: user 2min 55s, sys: 4.28 s, total: 2min 59s
Wall time: 1min 44s


## TF-IDF

Vamos usar TF-IDF para extrair uma lista de até 5 keywords de cada descrição de produto. Essa lista será utilizada como filtro para o cálculo de similaridade, pois iremos calcular a similaridade apenas entre items que tenham ao menos uma keyword compartilhada.

Levando em consideração que:
- O cálculo de similaridade é demorado e custoso para ser reaizado na base inteira
- Uma operação entre dois sets de keywords é menos custosa
- Itens que não compartilhem nenhuma keyword terão provavelmente uma similaridade baixa entre si
Vamos utilizar esse filtro para reduzir o processamento total no cálculo da similaridade

In [49]:
tf_idf = TfidfVectorizer(smooth_idf=True,use_idf=True)
tf_idf.fit(pd.concat([df_test["normalized_text"], df["normalized_text"]]))
feature_names = tf_idf.get_feature_names_out()

In [50]:
tfidf_vector_test = tf_idf.transform(df_test["normalized_text"])
tfidf_vector = tf_idf.transform(df["normalized_text"])

In [55]:
# extracting keywords
keywords = []
for i in range(df_test.shape[0]):
    keywords.append(extract_top_n(feature_names, tfidf_vector[i]).keys())

df_test["keywords"] = keywords

In [56]:
# extracting keywords
keywords = []
for i in range(df.shape[0]):
    keywords.append(extract_top_n(feature_names, tfidf_vector[i]).keys())

df["keywords"] = keywords

In [58]:
%%time
# creating a comparable list from items of `df` for each item of `df_test`
def apply_filter(row):
    df["filter"] = df.apply(lambda x: len(x["keywords"] & row["keywords"]), axis=1)
    return list(df[df["filter"]>0].index.values)

df_test["comparable"] = df_test.apply(lambda x: apply_filter(x), axis=1)

In [59]:
df_test.head()

,ITE_ITEM_TITLE,normalized_text,keywords,comparable
0,Tênis Olympikus Esporte Valente - Masculino Kids,tênis olympikus esporte valente masculino kid,"(posh, ascension, vermelho, preto, masculino)","[0, 5, 19, 30, 31, 35, 44, 49, 51, 85, 87, 90,..."
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,bicicleta barra forte samy c marchas cubo c ro...,"(spider, levinho, super, corrida, caminhada)","[1, 20, 21, 30, 55, 56, 64, 108, 121, 132, 135..."
2,Tênis Usthemp Slip-on Temático - Labrador 2,tênis usthemp slip on temá tico labrador,"(parc, ice, le, hock, black)","[2, 12, 134, 195, 341, 364, 439, 473, 492, 515..."
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tênis casual feminino moleca tecido tie dye,"(triunfo, tendência, nova, esportivo, academia)","[3, 7, 21, 30, 44, 49, 65, 78, 81, 106, 108, 1..."
4,Tênis Star Baby Sapatinho Conforto + Brinde,tênis star baby sapatinho conforto brinde,"(tauda, carregáve, inteligente, usb, bicicleta)","[4, 16, 52, 54, 176, 191, 338, 350, 368, 374, ..."


## Similaridade

Calculando a similaridade no dataset `df_test` com os itens cmparáveis do dataset `df`

In [100]:
%%time
df_test["ITE_ITEM_TITLE_similar"], df_test["Score Similitud"] = zip(*df_test.apply(lambda x: most_similar(x, embeddings_test, embeddings, df), axis=1))

CPU times: user 1h 30min 3s, sys: 53.6 s, total: 1h 30min 57s
Wall time: 3h 22min 4s


In [101]:
df_test.head()

,ITE_ITEM_TITLE,normalized_text,keywords,comparable,ITE_ITEM_TITLE_similar,Score Similitud
0,Tênis Olympikus Esporte Valente - Masculino Kids,tênis olympikus esporte valente masculino kid,"(posh, ascension, vermelho, preto, masculino)","[0, 5, 19, 30, 31, 35, 44, 49, 51, 85, 87, 90,...",[Tênis Infantil Olympikus Valente Kids Masculi...,"[0.9, 0.9]"
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,bicicleta barra forte samy c marchas cubo c ro...,"(spider, levinho, super, corrida, caminhada)","[1, 20, 21, 30, 55, 56, 64, 108, 121, 132, 135...",[Sapatilha Neoprene Ciclismo Bike Caminhada Co...,"[0.6, 0.6]"
2,Tênis Usthemp Slip-on Temático - Labrador 2,tênis usthemp slip on temá tico labrador,"(parc, ice, le, hock, black)","[2, 12, 134, 195, 341, 364, 439, 473, 492, 515...","[Tênis Hocks Tempus Nuts, Tênis Hocks - On Cam...","[0.8, 0.8]"
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tênis casual feminino moleca tecido tie dye,"(triunfo, tendência, nova, esportivo, academia)","[3, 7, 21, 30, 44, 49, 65, 78, 81, 106, 108, 1...",[Tênis Feminino Esportivo Nexx 135 Cadarço Tec...,"[0.8, 0.8]"
4,Tênis Star Baby Sapatinho Conforto + Brinde,tênis star baby sapatinho conforto brinde,"(tauda, carregáve, inteligente, usb, bicicleta)","[4, 16, 52, 54, 176, 191, 338, 350, 368, 374, ...",[Tênis De Led Infantil Recarregável Masculino ...,"[0.6, 0.6]"


In [102]:
df_test[["ITE_ITEM_TITLE", "ITE_ITEM_TITLE_similar", "Score Similitud"]] \
    .explode(["ITE_ITEM_TITLE_similar", "Score Similitud"]) \
    .to_csv(processed_data, index=False)